STEP 1: Build the Heston Model

In [1]:
import numpy as np
import pandas as pd
import sys
import math
import matplotlib as mpl
from scipy.optimize import brute, fmin

1. We define f

In [2]:
i = complex(0, 1)

# To be used in the Heston pricer
def fHeston(s, St, K, r, T, sigma, kappa, theta, volvol, rho):
    # To be used a lot
    prod = rho * sigma * i * s

    # Calculate d
    #d1 = (prod - kappa)**2
    #d2 = (sigma**2) * (i * s + s**2)
    #d = np.sqrt(d1 + d2)
    d = np.sqrt(prod**2-(sigma**2)*(2*i*s-s**2))

    # Calculate g
    g1 = kappa - prod - d
    g2 = kappa - prod + d
    g = g1 / g2

    # Calculate first exponential
    #exp1 = np.exp(np.log(St) * i * s) * np.exp(i * s * r * T)
    exp1 = (St ** (i * s)) * np.exp(i * s * r * T)
    exp2 = 1 - g * np.exp(-d * T)
    exp3 = 1 - g
    mainExp1 = exp1 * np.power(exp2 / exp3, (-2 * theta * kappa) / (sigma**2))

    # Calculate second exponential
    exp4 = theta * kappa * T / (sigma**2)
    exp5 = volvol / (sigma**2)
    exp6 = (1 - np.exp(-d * T)) / (1 - g * np.exp(-d * T))
    mainExp2 = np.exp((exp4 * g1) + (exp5 * g1 * exp6))

    return (mainExp1 * mainExp2)


2. We calculate integral

In [3]:
# Heston Pricer
def priceHestonMid(St, K, r, T, sigma, kappa, theta, volvol, rho):
    P, iterations, maxNumber = 0, 1000, 100
    ds = maxNumber / iterations
    element1 = 0.5 * (St - K * np.exp(-r * T))
    # Calculate the complex integral
    # Using j instead of i to avoid confusion
    for j in range(1, iterations):
        s1 = ds * (2 * j + 1) / 2
        s2 = s1 - i
        #numerator1 = fHeston(s2, St, K, r, T, sigma, kappa, theta, volvol, rho) 
        numerator1 = np.exp(r * T) * fHeston(s2, St, K, r, T, sigma, kappa, theta, volvol, rho)
        numerator2 = K * fHeston(s1, St, K, r, T, sigma, kappa, theta, volvol, rho)
        denominator = np.exp(np.log(K) * i * s1) * i * s1
        P += ds * (numerator1 - numerator2) / denominator
    element2 = P / np.pi
    return np.real((element1 + element2))

Now, let's calculating option Greeks. We are only using European options. 

In [6]:
def get_delta(s, k, rf, vol, t):
    d1 = (math.log(s/k) + (rf +  math.pow(vol, 2)/2 ) * t ) / (vol * math.sqrt(t) )
    d2 = d1 - vol * math.sqrt(t)
    delta = norm.cdf(d1)
    return delta

In [ ]:
def get_gamma(s, k, rf, vol, t,ds = 0.01):
    s += ds
    delta = get_delta(s, k, rf, vol, t)
    after_delta = delta
    s -= ds
    get_price_delta(s, k, rf, vol, t)
    orig_delta = delta
    gamma = (after_delta - orig_delta) / ds
    return gamma

In [ ]:
def get_theta(s, k, rf, vol, t, dt = 0.0027777):
    t += dt
    after_price = priceHestonMid() #heston model price 
    t -= dt
    orig_price = priceHestonMid() #heston model price 
    theta = (after_price - orig_price) / dt#* (-1)
    return theta